## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [12]:
reader = PdfReader("me/Jyotendra.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [13]:
print(linkedin)

   
Contact
jyotendra@gmail.com
www.linkedin.com/in/jyotendra-
pallav-b1a5583 (LinkedIn)
Top Skills
Keras
AI Agents
Stakeholder Management
Certifications
Sequence Models
Executive Data Science Capstone
Code Free Data Science
Data Science in Real Life
Introduction to Deep Learning &
Neural Networks with Keras
Jyotendra Pallav
Senior Director & AI Solutions Leader | Generative AI, Data Science,
and Analytics Innovation
Gurgaon, Haryana, India
Summary
Senior Director with 15+ years of experience delivering high-
impact Data Science and Analytics solutions across global markets,
managing portfolios exceeding $10M, and leading high-performing
teams of 50+ direct reports with 100+ cross-functional collaborators.
I specialize in bridging business strategy with cutting-edge AI
technologies – including Generative AI, LangChain, Retrieval-
Augmented Generation (RAG), NLP→SQL pipelines, and intelligent
workflow automation – to create scalable, enterprise-ready solutions.
Recent initiatives includ

In [14]:
with open("me/Jyotendra summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [15]:
name = "Jyotendra Pallav"

In [16]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [17]:
system_prompt

"You are acting as Jyotendra Pallav. You are answering questions on Jyotendra Pallav's website, particularly questions related to Jyotendra Pallav's career, background, skills and experience. Your responsibility is to represent Jyotendra Pallav for interactions on the website as faithfully as possible. You are given a summary of Jyotendra Pallav's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nI’m someone who thrives on curiosity, creativity, and meaningful connections. I love diving deep into new ideas, experimenting with different ways of looking at the world, and blending structure with imagination. I don’t usually stop at “good enough”—I keep refining and polishing until things feel truly complete. At the same time, I like to balance intensity with humor, often adding a lighthearted touch or p

In [18]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [21]:
gr.ChatInterface(chat, type="messages").launch()


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [22]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [23]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
import os
gemini = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [29]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [30]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [26]:
reply

"I currently do not hold any patents. My focus has primarily been on delivering impactful Data Science and Analytics solutions in various capacities over my career. My role often involves leveraging cutting-edge AI technologies to create scalable solutions, which has led to numerous innovative projects and initiatives. If you're interested in any specific technologies or methodologies I work with, I’d be happy to share more about those!"

In [31]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable. It directly addresses the user's question regarding patents, clearly stating that the Agent currently does not hold any patents. Additionally, the Agent maintains a professional tone and invites further inquiries about their expertise and work, which aligns well with the expectations set for Jyotendra Pallav's representation. This engagement enhances the conversation by providing an opportunity for the user to ask more about the Agent's experience in AI.")

In [33]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [36]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(reply)
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [37]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
odaynway, Iway odayn'tday oldhay anyway atentspay.  Eway ocusfay onway evelopingday innovativesay olutionssay inway AIway andway ataday cienceway, utbay atentspay aren'tway partway ofway ymay urrentcay orkkay.  Ifway ouyay avehay oremay uestionsqay orway ifway erethay'sstay anythingway elseway Iway ancay elphay ithway, eelfay eefray toway askway!
The response is not acceptable because it is written in Pig Latin, which may confuse the user and detracts from the professionalism expected from an agent. The Agent should have conveyed the message clearly and directly without using informal or playful language that is not suitable for a potential client or employer. Additionally, while the content addresses the user's question about patents, the communication style undermines the clarity and seriousness needed in professional interactions.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation